In [ ]:
from main.core.layers.gold import Gold
from main.core.layers.silver import Silver
from datetime import date
import pandas as pd


class CnesEstabelecimentosMetrics(Gold):
    def __init__(self, year_month: str = "all"):
        super().__init__(name="cnes_estabelecimentos_metrics")
        self.year_month = year_month

        # entradas
        self.inputs = {
            "estabelecimentos": self.read_silver_parquet("cnes_estabelecimentos", year_month=self.year_month),
            "populacao": self._read_single_parquet(fs_client=self._gold_fs, path="populacao/data.parquet"),
        }

    def definition(self) -> pd.DataFrame:
        estab = self.inputs["estabelecimentos"].copy()
        pop = self.inputs["populacao"].copy()

        # ============================================================
        # filtros
        # ============================================================
        mask_sus = estab.get("TP_SUS_NAO_SUS", "").eq("S")
        mask_med = estab.get("DS_ATIVIDADE_PROFISSIONAL", "").astype(str).str.startswith("MEDICO", na=False)
        estab = estab[mask_sus & mask_med]

        keep_cols = [
            "CO_PROFISSIONAL_SUS",
            "NO_MUNICIPIO",
            "DS_ATIVIDADE_PROFISSIONAL",
            "TP_SUS_NAO_SUS",
            "CO_MUNICIPIO",
            "year_month",
        ]
        estab = estab[[c for c in keep_cols if c in estab.columns]].copy()

        # tipos e chaves
        estab["CO_MUNICIPIO_SEM_DIGITO"] = pd.to_numeric(estab["CO_MUNICIPIO"], errors="coerce").astype("Int64")
        estab["YYYY"] = estab["year_month"].astype(str).str[:4].astype("Int16")
        estab["MM"] = estab["year_month"].astype(str).str[4:6]
        estab["MM"] = pd.Categorical(estab["MM"], categories=[f"{m:02d}" for m in range(1, 13)], ordered=True)

        # agrega profissionais únicos
        group_cols = ["CO_MUNICIPIO_SEM_DIGITO", "NO_MUNICIPIO", "DS_ATIVIDADE_PROFISSIONAL", "TP_SUS_NAO_SUS", "YYYY", "MM"]
        g = (
            estab.groupby(group_cols, dropna=False)["CO_PROFISSIONAL_SUS"]
            .nunique()
            .reset_index(name="TOTAL_PROFISSIONAIS")
        )

        # normaliza população
        for c in ["CO_MUNICIPIO_SEM_DIGITO", "YYYY", "MM"]:
            if c not in pop.columns:
                raise KeyError(f"população: coluna obrigatória ausente: {c}")

        pop = pop.copy()
        pop["CO_MUNICIPIO_SEM_DIGITO"] = pd.to_numeric(pop["CO_MUNICIPIO_SEM_DIGITO"], errors="coerce").astype("Int64")
        pop["YYYY"] = pd.to_numeric(pop["YYYY"], errors="coerce").astype("Int16")
        pop["MM"] = pop["MM"].astype(str).str.zfill(2)

        # join e métrica
        join_keys = ["CO_MUNICIPIO_SEM_DIGITO", "YYYY", "MM"]
        cols_pop = join_keys + [
            "CO_UF", "NO_UF", "NO_REGIAO", "NO_MUNICIPIO_IBGE",
            "POPULACAO_MENSAL", "POPULACAO", "GROWTH_ABS", "GROWTH_PCT"
        ]
        cols_pop = [c for c in cols_pop if c in pop.columns]

        df = g.merge(pop[cols_pop], on=join_keys, how="left")
        df["PROFISSIONAIS_POR_1000"] = (
            (df["TOTAL_PROFISSIONAIS"] / df["POPULACAO_MENSAL"].replace({0: pd.NA})) * 1000
        )

        df["DATA_INGESTAO"] = pd.Timestamp.today().strftime("%Y-%m-%d")
        df["YYYYMM"] = df["YYYY"].astype(str) + df["MM"].astype(str)

        return df


# Teste
m = CnesEstabelecimentosMetrics(year_month="202401")
df = m.definition()
print(df.shape)
df.head()


(14106857, 17)


,CO_UNIDADE,CO_PROFISSIONAL_SUS,NO_PROFISSIONAL,CO_CBO,TP_SUS_NAO_SUS,DS_ATIVIDADE_PROFISSIONAL,NO_FANTASIA,NO_BAIRRO,NO_MUNICIPIO,CO_MUNICIPIO,CO_SIGLA_ESTADO,CO_CEP,ds_localidade,SK_REGISTRO,DATA_INGESTAO,YYYYMM,DUMMY_COLUMN
0,SP00003509205000001329730000101,F3575C9617F8998A,WLADMIR GUBEISSI PINTO FILHO,225250,S,MEDICO GINECOLOGISTA E OBSTETRA,CLINICA MEDICA SULLA PELLE S C LTDA,ACLIMACAO,SAO PAULO,355030,SP,01530000,"01530000,SAO PAULO,SP,Brasil",SP00003509205000001329730000101_F3575C9617F899...,2025-11-03,202101,1
1,SP00003509205000001329730000101,9D27061F6644A854,MARTHA TIDORI KIOTA KOTSUBO,225320,S,MEDICO EM RADIOLOGIA E DIAGNOSTICO POR IMAGEM,CLINICA MEDICA SULLA PELLE S C LTDA,ACLIMACAO,SAO PAULO,355030,SP,01530000,"01530000,SAO PAULO,SP,Brasil",SP00003509205000001329730000101_9D27061F6644A8...,2025-11-03,202101,1
2,3500100047406,F6965A7E959C6A39,GRAZIELE DAVID,251510,N,PSICOLOGO CLINICO,GRAZIELE DAVID PSICOLOGA EIRELI,CENTRO,ADAMANTINA,350010,SP,17800000,"17800000,ADAMANTINA,SP,Brasil",3500100047406_F6965A7E959C6A39_251510,2025-11-03,202101,1
3,3500100081655,1FC46A4EA7312E5B,KELLY PRESTES RUFINO,223605,N,FISIOTERAPEUTA GERAL,RUFINO PRESTES LTDA,CENTRO,ADAMANTINA,350010,SP,17800000,"17800000,ADAMANTINA,SP,Brasil",3500100081655_1FC46A4EA7312E5B_223605,2025-11-03,202101,1
4,3500100109789,EDEB3090A41EB3A5,JOANA DARC BORRO,251510,S,PSICOLOGO CLINICO,JOANA DARC BORRO,VILA CICMA,ADAMANTINA,350010,SP,17800000,"17800000,ADAMANTINA,SP,Brasil",3500100109789_EDEB3090A41EB3A5_251510,2025-11-03,202101,1
